In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import random
driver = webdriver.Chrome(executable_path='C:\chromedriver\chromedriver.exe')

In [2]:
##HELPERFUNCTIONS

def get_matchLinks(page):
    matchesLinks = []
    ## https://www.hltv.org/results?offset=0
    driver.get('https://www.hltv.org/results?offset='+ str(page) +'&stars=1')
    content = driver.page_source
    soup = BeautifulSoup(content)

    ## FIND MATCHES
    for div in soup.findAll('div', attrs={'class': 'results'}):
        for a in div.findAll('a', attrs={'class': 'a-reset'}):
            link = a['href']
            if (link[:8] == "/matches"):
                matchesLinks.append(link)
    return matchesLinks

def get_page_data(link):
    url = 'https://www.hltv.org/' + str(link)
    driver.get(url)
    content = driver.page_source
    soup = BeautifulSoup(content)    
    return soup

def get_game_links(soup):
    statlinks = []
    for div in soup.findAll('div', attrs={'class': 'results-center-stats'}):
        for a in div.findAll('a'):
            statlinks.append(a['href'])
    return statlinks

def get_game_stats(link):

    page = get_page_data(link)
    game_stats = {}

    scoreboard = page.findAll('table', {'class':'stats-table'})
    game_stats['Scoreboard'] = pd.read_html(str(scoreboard))

    misc_stats = page.findAll('div', attrs={'class': 'right'})
    game_stats['Starting side'] = misc_stats[0].contents[4]['class'][0][:-6]
    game_stats['Rounds'] = misc_stats[0].text
    game_stats['Team rating'] = misc_stats[1].text
    game_stats['Entries'] = misc_stats[2].text
    game_stats['Clutches'] = misc_stats[3].text

    return game_stats

In [3]:
##GET SOUP STORE SOUP
matchesLinks = get_matchLinks(0)
soup = get_page_data(matchesLinks[0])

In [4]:
##BASIC INFO

##GET DATE from https://www.hltv.org/matches/2347834***
names = ["Date","Tournament","Team1","Result1","Team2","Result2","Map","MapID","StatLinks","Players"]
data = []
for div in soup.findAll('div', attrs={'class': 'match-page'}):
    pageDate = div.find('div', attrs={'class': 'date'})
    pageTournament = div.find('div', attrs={'class': 'event text-ellipsis'})
    data.append(pageDate.text)
    data.append(pageTournament.text)

##GET FINALRESULT1 from https://www.hltv.org/matches/2347834***
for div in soup.findAll('div', attrs={'class': 'team1-gradient'}):
    pageTeam1 = div.find('div', attrs={'class': 'teamName'})
    pageResult1 = div.find('div', attrs={'class': ['won', 'lost', 'tie']})
    data.append(pageTeam1.text)
    data.append(pageResult1.text)

##GET FINALRESULT2 from https://www.hltv.org/matches/2347834***
for div in soup.findAll('div', attrs={'class': 'team2-gradient'}):
    pageTeam2 = div.find('div', attrs={'class': 'teamName'})
    pageResult2 = div.find('div', attrs={'class': ['won', 'lost', 'tie']})
    data.append(pageTeam2.text)
    data.append(pageResult2.text)

maps = []
mapsID = []
players = []

##GET MAPS AND MAPSID from https://www.hltv.org/matches/2347834***
for div in soup.findAll('div', attrs={'class': 'stats-menu-link'}):
    map1 = div.findAll('div', attrs= {'class': 'dynamic-map-name-full'})
    for div in map1:
        maps.append(div.text)
        mapsID.append(div['id'])
data.append(maps)
data.append(mapsID)

##GET DETAILED STATS LINKS
data.append(get_game_links(soup))

##GET PLAYER INFO
for div in soup.findAll('div', attrs={'class': 'lineups'}):
    for table in div.findAll('td',attrs={'class': 'player player-image'}):
        for a in table.findAll('a'):
            players.append(a['href'])
data.append(players)
    
result = [[names],[data]]
for i in range(0,len(names)):
    print(names[i] + " : " + str(data[i]))

Date : 11th of April 2021
Tournament : ESL Pro League Season 13
Team1 : Gambit
Result1 : 2
Team2 : Heroic
Result2 : 3
Map : ['All maps', 'Inferno', 'Vertigo', 'Train', 'Overpass', 'Mirage']
MapID : ['all', '117935', '117943', '117951', '117960', '117962']
StatLinks : ['/stats/matches/mapstatsid/117935/gambit-vs-heroic', '/stats/matches/mapstatsid/117943/heroic-vs-gambit', '/stats/matches/mapstatsid/117951/gambit-vs-heroic', '/stats/matches/mapstatsid/117960/heroic-vs-gambit', '/stats/matches/mapstatsid/117962/heroic-vs-gambit']
Players : ['/player/8528/hobbit', '/player/15071/interz', '/player/16555/ax1le', '/player/16920/sh1ro', '/player/16921/nafany', '/player/7964/cadian', '/player/10610/refrezh', '/player/10994/stavn', '/player/12018/teses', '/player/14148/sjuush']


In [5]:
links = data[8]
# Match Data
match = {}
count = 0
for link in links:
    count += 1
    scoreboard = get_game_stats(link)
    match['Game ' + str(count)] = scoreboard

    sleeptime = random.uniform(1, 2) # Stealth mode = engaged
    time.sleep(sleeptime)

In [6]:
match

{'Game 1': {'Scoreboard': [   Gambit   K (hs)   A (f)   D   KAST  K-D Diff   ADR  FK Diff  Rating2.0
   0   sh1ro   32 (7)   4 (3)  19  86.1%        13  80.2        2       1.38
   1  Hobbit  24 (13)  11 (1)  22  77.8%         2  85.2        3       1.25
   2   Ax1Le  21 (11)   6 (0)  23  63.9%        -2  73.4        6       1.08
   3  nafany  21 (11)   5 (1)  26  58.3%        -5  71.0       -5       0.87
   4  interz  19 (12)   2 (0)  23  69.4%        -4  51.2       -2       0.84,
       Heroic   K (hs)   A (f)   D   KAST  K-D Diff   ADR  FK Diff  Rating2.0
   0    stavn  29 (12)   5 (2)  24  69.4%         5  80.7        4       1.21
   1  refrezh  22 (10)   9 (5)  21  61.1%         1  66.0       -2       1.01
   2   sjuush   21 (8)  11 (2)  23  66.7%        -2  77.8       -1       0.99
   3   cadiaN   20 (3)   4 (3)  24  69.4%        -4  57.8        0       0.88
   4    TeSeS  19 (17)   5 (2)  25  66.7%        -6  58.4       -5       0.83],
  'Starting side': 'ct',
  'Rounds': '17 : 